#### Load Data

# Import Movie Review Data

Import the dataset as pandas dataframe

In [0]:
import pandas as pd

Data can be downloaded from Kaggle at the following URL

- https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [0]:
df = pd.read_csv('labeledTrainData.tsv.zip',header=0, delimiter="\t", quoting=3)

Split Data into Training and Test Data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'],
    df['sentiment'],
    test_size=0.2, 
    random_state=42
)

# Build the Tokenizer

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

c:\users\ibm_admin\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [0]:
top_words = 10000

In [0]:
t = Tokenizer(num_words=top_words) # num_words -> Vocablury size

In [0]:
t.fit_on_texts(X_train.tolist())

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [0]:
X_train = t.texts_to_sequences(X_train.tolist())

In [0]:
X_test = t.texts_to_sequences(X_test.tolist())

How many words in each review?

# Pad Sequences - Important

In [0]:
from tensorflow.python.keras.preprocessing import sequence

In [0]:
max_review_length = 300

In [0]:
X_train = sequence.pad_sequences(X_train,maxlen=max_review_length,padding='post')

In [0]:
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post')

# Build Embedding Matrix from Pre-Trained Word2Vec

Load pre-trained Gensim Embeddings

In [0]:
import gensim

c:\users\ibm_admin\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [0]:
word2vec = gensim.models.Word2Vec.load('word2vec-movie-50')

Embedding Size

In [0]:
embedding_vector_length = word2vec.wv.syn0.shape[1]

Build matrix for current data

In [0]:
embedding_matrix = np.zeros((top_words + 1, embedding_vector_length))

In [0]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > top_words:
        break
    if word in word2vec.wv.vocab:
        embedding_vector = word2vec.wv[word]
        embedding_matrix[i] = embedding_vector

# Build the Graph

In [0]:
from tensorflow.python.keras.models import Sequential

In [0]:
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten, LSTM

In [0]:
model = Sequential()

Add Embedding layer
 - Embedding Layer Input = Batch_Size * Length of each review

In [0]:
model.add(Embedding(top_words + 1,
                    embedding_vector_length,
                    input_length=max_review_length,
                   weights=[embedding_matrix],
                   trainable=False)
         )

Embedding Layer Output - 
[Batch_Size , Review Length , Embedding_Size]

Add Layer with 100 LSTM Memory Units

In [0]:
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#outout is last hidden state
#batch_size, 100

In [0]:
model.add(Dense(1,activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# Execute the graph

In [0]:
model.fit(X_train,y_train,
          epochs=10,
          batch_size=128,          
          validation_data=(X_test, y_test),
         verbose=1)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
19968/20000 [============================>.] - ETA: 0s - loss: 0.6629 - acc: 0.5941